# Visualize Notebook
In this notebook the Gold Dataframe will be read and extract information of it. The objective is to see the correlations between the variables and the GDP and also what countries have the highest correlation value.


## Imports
Start importing all the libraries and also the methods of pvalue and search indicators that will be used later in the notebook.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro
import os
import statistics
import seaborn as sns
from scipy.stats import norm

import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact, interact_manual
from Project.Utils.visualize import pvalue, search_indicators, search


## Correlation dataframe.
This dataframe is the main piece of the notebook. Consists in generating for every country the correlation matrix for it and saving only the correlation value of the different variables with the GDP. Later on is concatenated and generates the following result:

In [2]:
#One dataframe per country

df= pd.read_csv (os.getcwd()+'/Output/GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = ['Country']
aux_df = pd.DataFrame()

#List all the countries, none repeated
countries = set(df['Country'].to_list())

country_dict = {}
corr_dict = {}

for country in countries:

    #Get the DataFrame for a given country
    country_df = df.loc[df['Country'] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {'GDP': country}).drop(index = ['Year', 'GDP'])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format and show it
corr_df = corr_df.transpose()
corr_df


,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Lebanon,-0.503101,NaN,NaN,0.891247,-0.939681,-0.122113,0.062329,NaN,0.888425,0.146781,...,-0.145843,-0.985057,0.890831,0.634427,0.896726,0.899102,NaN,NaN,NaN,0.958025
Mongolia,-0.835522,NaN,NaN,0.925688,-0.860031,0.865737,-0.936540,-0.923997,0.968864,-0.891779,...,0.925761,-0.919030,-0.779568,-0.679556,-0.795143,0.970238,NaN,NaN,NaN,0.938037
Chile,-0.837463,NaN,NaN,0.953282,-0.915442,-0.866672,0.858393,NaN,0.959914,-0.955864,...,0.102632,-0.911129,0.292348,-0.335626,0.877975,0.029696,0.793805,0.335628,0.985091,0.918629
San Marino,0.065850,NaN,NaN,NaN,NaN,0.357578,-0.162883,NaN,NaN,NaN,...,0.333934,0.001470,NaN,NaN,-0.198150,NaN,NaN,NaN,NaN,-0.004386
Singapore,-0.892755,0.771369,NaN,0.962109,-0.946267,-0.876853,0.801086,-0.876330,0.975188,-0.964077,...,-0.444154,NaN,-0.353507,0.866337,0.958775,-0.864015,NaN,NaN,NaN,0.990639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Uruguay,-0.576932,0.890955,NaN,0.858527,-0.828063,-0.837514,0.651712,0.696210,0.907707,-0.816982,...,0.837996,-0.911606,-0.291775,0.457307,0.819461,-0.823795,NaN,NaN,NaN,0.899639
Myanmar,-0.967870,NaN,NaN,NaN,-0.180847,-0.973266,-0.978075,0.757785,0.960719,-0.954086,...,-0.544811,-0.941386,0.932649,0.951070,0.726087,0.899596,NaN,NaN,NaN,0.947493
Austria,-0.828536,NaN,0.84706,0.938655,-0.780851,-0.146353,-0.147189,-0.074849,0.933147,-0.672080,...,0.145298,0.842808,-0.356982,-0.406429,0.767288,-0.902411,0.172537,-0.544896,0.901465,0.789386
Comoros,-0.592755,NaN,NaN,NaN,NaN,-0.894111,-0.928384,NaN,0.919907,-0.944990,...,-0.698759,-0.669837,0.469369,0.268741,-0.530582,-0.885866,NaN,NaN,NaN,0.929551


In [3]:
import ipywidgets as widgets
import plotly.express as px


indicator = widgets.SelectMultiple(
    options = corr_df.columns.tolist(),
    value = ['AgriShareGDP'],
    description='Indicator',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)


def globalGrapgh(indicator):
    fig = px.choropleth(corr_df, locations = corr_df.index, locationmode='country names', 
                        color= indicator[0],projection="natural earth",
                        color_continuous_scale='Reds',
                    width=700, height=500)
    fig.update(layout_coloraxis_showscale=True)
    fig.show()
    
    
widgets.interactive(globalGrapgh, indicator = indicator) 

interactive(children=(SelectMultiple(description='Indicator', index=(0,), layout=Layout(height='80px', width='…

In [4]:
def tableOut(Threshold, Country):
    
    """ df = search_indicators(Threshold, corr_df)[Country]
    df  = pd.merge(df, pvalue(Country), left_index=True, right_index=True, how = 'inner')
    
    
    left = pd.Series([0.05], index=['P-value'])
    left2 = pd.Series([-1], index=['GDP Correlation'])
    left3 = pd.Series([0], index=['GDP Correlation'])
    
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Correlation'])\
        .format('{:,.12f}', subset = ['P-value'], ) """

    display(search(Threshold, corr_df))
    

@interact(
    Country = sorted(corr_df.index.tolist()),
    Threshold = (0, 1, 0.05))
def g(Country = 'Afghanistan', Threshold = 0.7):
    return tableOut(Threshold,Country)

    
        

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…